<a href="https://colab.research.google.com/github/Inderivator/PlaylistSorter/blob/Playlist-sorter/AI_Playlist_Sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>